In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as p
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import colors

from itertools import combinations
import scipy
from scipy.ndimage.filters import gaussian_filter1d
import sys
import os
import copy
atish_assay_path = 'fitness_assay_grantedits.py'
sys.path.append(os.path.dirname(os.path.expanduser(atish_assay_path)))
import fitness_assay_grantedits as atish

tools_path = '../code/tools.py'
sys.path.append(os.path.dirname(os.path.expanduser(tools_path)))
import tools

sns.set_style('white')
sns.set_style('ticks')
sns.set_color_codes()
from tools import mutant_colorset

In [2]:
cov_thresh = 1e6

fitness_data = p.read_csv('../data/DoubleBC_Merged_Fitness_Atish_Weighted_Default_AllConditions_IncludingOld_swapsremoved.csv')

merged_fitness_data = fitness_data

In [4]:
this_data = merged_fitness_data
this_data = this_data.replace([np.inf, -np.inf], np.nan)
# this_data = this_data.dropna('columns',how='all')
this_data = this_data.dropna()
this_data = this_data.sort_values('barcode')

In [6]:
old_conditions = {
                  '3':['3.1','3.2','3.3'],
                  '6':['6.1','6.2','6.3'],
                  '13':['13.1','13.2','13.3'],
                  '18':['18.1','18.2','18.3'],
                  '20':['20.1','20.2','20.3'],
                  '21':['21.1','21.2','21.3'],
                  '23':['23.1','23.2','23.3']
                 }

old_conditions = {
                  '3':['t3.1','t3.2','t3.3'],
                  '6':['t6.1','t6.2','t6.3'],
                  '13':['t13.1','t13.2','t13.3'],
                  '18':['t18.1','t18.2','t18.3'],
                  '19':['t19.1','t19.2','t19.3'],
                  '20':['t20.1','t20.2','t20.3'],
                  '21':['t21.1','t21.2','t21.3'],
                  '23':['t23.1','t23.2','t23.3'],
                  'M3_Batch_3':['3.1','3.2','3.3'],
                  'M3_Batch_6':['6.1','6.2','6.3'],
                  'M3_Batch_13':['13.1','13.2','13.3'],
                  'M3_Batch_18':['18.1','18.2','18.3'],
#                   'M3_Batch_19':['t19.1','t19.2','t19.3'],
                  'M3_Batch_20':['20.1','20.2','20.3'],
                  'M3_Batch_21':['21.1','21.2','21.3'],
                  'M3_Batch_23':['23.1','23.2','23.3'],
                  'Ferm_44hr_Transfer':['t14.1','t14.2','t14.3'],
                  'Ferm_40hr_Transfer':['t15.1','t15.2','t15.3'],
                  'Ferm_54hr_Transfer':['t16.1','t16.2','t16.3'],
                  'Ferm_50hr_Transfer':['t17.1','t17.2','t17.3'],
                  'Resp_24hr_Transfer':['t5.1','t5.2','t5.3'],
                  'Resp_3Day_Transfer':['t7.1','t7.2','t7.3'],
                  'Resp_4Day_Transfer':['t8.1','t8.2'], # 8.3 has too little data
                  'Resp_5Day_Transfer':['t25.1','t25.2','t25.3'],
                  'Resp_6Day_Transfer':['t26.1','t26.2','t26.3'],
                  'Resp_7Day_Transfer':['t27.1','t27.2','t27.3'],
                  'DMSO':['t1.1','t1.2','t1.3'],
                  'Geldanamycin8.5uM':['t2.4','t2.5','t2.6'],
                    ## radicicol
                    ## benomyl
                    ## fluconazole
                    ## Glucose gradient
    
    }

bigbatch_conditions = {
                        '1BB_M3':['A','B','C','D'],
                        '1BB_Baffle':['E','F'],
                        '1BB_1.4%Gluc' :['G','H'],
                        '1BB_1.8%Gluc' :['I','J'],
                        '1BB_0.2MNaCl' :['K','L'],
#                         '1BB_0.2MNaCl' :['K'], 
                        '1BB_0.5MNaCl' :['M','N'],
                        '1BB_0.2MKCl' :['O'],
                        '1BB_0.5MKCl' :['P'],
                        '1BB_8.5uMGdA' :['Q','R'],
                        '1BB_17uMGdA' :['S','T'],
                        '1BB_2ugFlu' :['U','V'],
                        '1BB_0.5ugFlu' :['W','X'],
                        '1BB_1%Raf' :['Y','Z'],
                        '1BB_0.5%Raf' :['AA','BB'],
                        '1BB_1%Gly' :['CC','DD'],
                        '1BB_1%EtOH' :['EE','FF'],
                        '1BB_SucRaf' :['GG'],
                      }



conditions = {**old_conditions, **bigbatch_conditions}

just_reps = [rep for reps in conditions.values() for rep in reps]

In [10]:
this_data[['barcode']+[condition+'_fitness' for condition in conditions]].to_csv('~/Desktop/Fitness_Data_for_Lesha.csv',index=False)

In [11]:
this_data[['barcode']+[condition+'_fitness' for condition in conditions]]

,barcode,3_fitness,6_fitness,13_fitness,18_fitness,19_fitness,20_fitness,21_fitness,23_fitness,M3_Batch_3_fitness,...,1BB_0.5MKCl_fitness,1BB_8.5uMGdA_fitness,1BB_17uMGdA_fitness,1BB_2ugFlu_fitness,1BB_0.5ugFlu_fitness,1BB_1%Raf_fitness,1BB_0.5%Raf_fitness,1BB_1%Gly_fitness,1BB_1%EtOH_fitness,1BB_SucRaf_fitness
0,53,0.342350,0.219980,0.329707,0.364880,0.342042,0.483977,0.390061,0.456529,0.360413,...,0.281774,0.301609,0.420630,0.235757,0.589379,0.097501,0.267181,0.347187,0.537630,0.381319
1,151,1.147750,1.718462,1.455814,1.191489,1.470287,0.985456,0.900421,1.077938,1.156822,...,-2.547008,0.686606,0.957988,1.027523,0.202130,1.039126,0.994910,1.038026,0.715633,0.595644
2,262,0.126882,0.158263,0.139936,0.191609,0.190583,0.397133,0.372354,0.457362,0.126707,...,0.301131,0.220915,0.410144,0.210633,0.585619,0.019456,0.286933,0.345255,0.493757,0.350548
3,273,0.793303,1.065206,1.061621,0.937571,1.066905,0.838311,0.805570,0.906270,0.800876,...,-1.036554,0.539668,0.692608,0.886732,0.235617,0.846375,0.775972,0.812373,0.494898,0.452001
4,323,0.826204,1.112005,0.957155,0.887126,0.960712,0.747896,0.805334,0.846050,0.833776,...,-0.430072,0.513107,0.625366,0.705413,0.285173,0.671028,0.628388,0.663332,0.433916,0.291819
8,689,1.171387,1.387260,1.170626,1.050745,1.181397,1.041810,0.889421,1.042369,1.180324,...,-3.315940,0.608307,0.930199,0.986992,0.016605,1.143336,0.986495,1.089611,0.688321,0.504585
9,697,0.309494,0.407635,0.194139,0.261355,0.257368,0.461606,0.359061,0.431557,0.313814,...,0.301192,0.349056,0.375616,0.249276,0.518481,0.137316,0.215339,0.299675,0.490181,0.369475
12,1379,1.111738,1.524418,1.316981,1.096624,1.343984,1.074118,0.926220,1.128933,1.119956,...,-2.072755,0.554624,0.936239,0.984122,-0.077199,0.980880,0.864478,0.983229,0.446068,0.416034
13,1488,1.140113,1.446139,1.188988,1.072210,1.255772,1.067751,0.802872,1.034957,1.148262,...,-3.708913,0.538311,0.887338,0.961630,-0.001119,1.092418,0.944628,1.035003,0.429861,0.561952
14,1564,0.274831,0.202017,0.210154,0.330708,0.223758,0.427730,0.461339,0.452582,0.287443,...,0.305997,0.302282,0.417557,0.212343,0.551510,-0.000640,0.178112,0.315670,0.506919,0.346503
